In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import imagenet_utils
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import time
import cv2
from tensorflow.keras.models import load_model
import pickle
from skimage import color
from skimage.feature import hog
import imutils

In [2]:
INPUT_SIZE = (500, 600)
PYR_SCALE = 1.5
WIN_STEP = 8
ROI_SIZE = (30, 55)
BATCH_SIZE = 64

In [3]:
def image_pyramid(image, scale=1.5, minSize=(224, 224)):
	# yield the original image
	yield image

	# keep looping over the image pyramid
	while True:
		# compute the dimensions of the next image in the pyramid
		w = int(image.shape[1] / scale)
		image = imutils.resize(image, width=w)

		# if the resized image does not meet the supplied minimum
		# size, then stop constructing the pyramid
		if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
			break

		# yield the next image in the pyramid
		yield image

In [4]:
def sliding_window(image, step, ws):
	# slide a window across the image
	for y in range(0, image.shape[0] - ws[1], step):
		for x in range(0, image.shape[1] - ws[0], step):
			# yield the current window
			yield (x, y, image[y:y + ws[1], x:x + ws[0]])


In [5]:
def classify_batch(model, batchROIs, batchLocs, labels,scaler, minProb=0.5,
    top=10, dims=(30, 55)):
    # pass our batch ROIs through our network and decode the
    # predictions
    batchROIs=scaler.transform(batchROIs)
    preds = model.predict(batchROIs)
#     P = imagenet_utils.decode_predictions(preds, top=top)
    # loop over the decoded predictions
    for i in range(0, preds.shape[0]):
            # filter out weak detections by ensuring the
            # predicted probability is greater than the minimum
            # probability
        if preds[i] > minProb:
                # grab the coordinates of the sliding window for
                # the prediction and construct the bounding box
            (pX, pY) = batchLocs[i]
            box = (pX, pY, pX + dims[0], pY + dims[1])

                # grab the list of predictions for the label and
                # add the bounding box + probability to the list
            L = labels.get(1, [])
            L.append((box, preds[i]))
            labels[1] = L

    # return the labels dictionary
    return labels

In [6]:
labels = {}

In [7]:
resized = cv2.imread("A:\\Infilect_project\\Product_detector\\Dataset\\ShelfImages\\train\\C1_P09_N2_S6_1.JPG")

In [8]:
(h, w) = resized.shape[:2]

In [9]:
print(resized.shape)

(2151, 3264, 3)


In [10]:
resized = cv2.resize(resized, INPUT_SIZE, interpolation=cv2.INTER_CUBIC)

In [11]:
batchROIs = None
batchLocs = []

In [12]:
def preprocessing_input_image(img, img_size):
    resized_image = np.resize(img, img_size)
    grayscale_image = color.rgb2gray(resized_image)
    features = hog(grayscale_image, 
                   orientations=30, 
                   pixels_per_cell=(16, 16), 
                   cells_per_block=(2, 2), 
                   block_norm='L2-Hys', 
                   visualize=False, 
                   transform_sqrt=False, 
                   feature_vector=True, 
                   multichannel=None)
    return features

In [13]:
train_scaler = pickle.load( open( "scalers//hog_product_scaler.pkl", "rb" ) )
product_classifier = load_model("models//hog_product_model.h5")
# for image in image_pyramid(resized, scale=PYR_SCALE,
#     minSize=ROI_SIZE):
for (x, y, roi) in sliding_window(resized, WIN_STEP, ROI_SIZE):
        # take the ROI and pre-process it so we can later classify the
        # region with Keras
    roi = img_to_array(roi)
    # 	roi = np.expand_dims(roi, axis=0)
    roi = preprocessing_input_image(roi, (128, 128))

        # if the batch is None, initialize it
    if batchROIs is None:
        batchROIs = roi

        # otherwise, add the ROI to the bottom of the batch
    else:
        batchROIs = np.vstack([batchROIs, roi])

        # add the (x, y)-coordinates of the sliding window to the batch
    batchLocs.append((x, y))

        # check to see if our batch is full
    if len(batchROIs) == BATCH_SIZE:
        # classify the batch, then reset the batch ROIs and
        # (x, y)-coordinates
        labels = classify_batch(product_classifier, batchROIs, batchLocs,
                labels, train_scaler,minProb=0.8)

            # reset the batch ROIs and (x, y)-coordinates
        batchROIs = None
        batchLocs = []

In [14]:
if batchROIs is not None:
	labels = classify_batch(product_classifier, batchROIs, batchLocs,
			labels, train_scaler,minProb=0.7)

In [18]:
# loop over the labels for each of detected objects in the image
for k in labels.keys():
  # clone the input image so we can draw on it
	clone = resized.copy()

	# loop over all bounding boxes for the label and draw them on
	# the image
	for (box, prob) in labels[k]:
		(xA, yA, xB, yB) = box
		cv2.rectangle(clone, (xA, yA), (xB, yB), (0, 255, 0), 2)

	# show the image *without* apply non-maxima suppression
	cv2.imshow("Without NMS", clone)
	clone = resized.copy()

	# grab the bounding boxes and associated probabilities for each
	# detection, then apply non-maxima suppression to suppress
	# weaker, overlapping detections
	boxes = np.array([p[0] for p in labels[k]])
	proba = np.array([p[1] for p in labels[k]])
#     print(proba)
	boxes = non_max_suppression(boxes, proba, overlapThresh=0.9)

	# loop over the bounding boxes again, this time only drawing the
	# ones that were *not* suppressed
	for (xA, yA, xB, yB) in boxes:
		cv2.rectangle(clone, (xA, yA), (xB, yB), (0, 0, 255), 2)

	# show the output image
	print("[INFO] {}: {}".format(k, len(boxes)))
	cv2.imshow("With NMS", clone)
	cv2.waitKey(0)

[INFO] 1: 1
